In [59]:
import pandas as pd
df = pd.read_csv('nhatot_crawl4ai.csv')
print(f"Số dòng ban đầu: {len(df)}")
# Bước 1: Loại bỏ các hàng mà tất cả các cột đều trống (toàn bộ là NaN)
df_cleaned = df.dropna(how='all')
print(f"Số dòng sau khi loại bỏ hàng trống hoàn toàn: {len(df_cleaned)}")
# Bước 2: Loại bỏ các hàng có quá nhiều giá trị thiếu
num_columns = len(df_cleaned.columns)
# Tính số lượng giá trị NaN trong mỗi hàng
missing_count = df_cleaned.isnull().sum(axis=1)
# Loại bỏ các hàng có số lượng giá trị NaN lớn hơn 6
threshold = 6
df_cleaned = df_cleaned[missing_count <= threshold]
print(f"Số dòng sau khi loại bỏ hàng có hơn {threshold} giá trị NaN: {len(df_cleaned)}")
df_cleaned.to_csv('cleaned_nhatot_data.csv', index=False)
print("Dữ liệu đã được làm sạch và lưu vào file 'cleaned_nhatot_data.csv'")

Số dòng ban đầu: 23527
Số dòng sau khi loại bỏ hàng trống hoàn toàn: 7151
Số dòng sau khi loại bỏ hàng có hơn 6 giá trị NaN: 7150
Dữ liệu đã được làm sạch và lưu vào file 'cleaned_nhatot_data.csv'


In [60]:
#xử lí hàng trùng:
file_path = "cleaned_nhatot_data.csv"
# Đọc file
df = pd.read_csv(file_path)
print("Số dòng ban đầu:", len(df))
# Xoá dòng trùng hoàn toàn (kể cả NaN)
df = df.drop_duplicates(keep="first")
print("Số dòng sau khi xoá trùng:", len(df))
# GHI ĐÈ LẠI FILE CŨ
df.to_csv(file_path, index=False)
print("✅ Đã cập nhật trực tiếp vào file:", file_path)
print(df.columns.tolist())



Số dòng ban đầu: 7150
Số dòng sau khi xoá trùng: 6396
✅ Đã cập nhật trực tiếp vào file: cleaned_nhatot_data.csv
['Giá bán', 'Thành phố', 'Phường/Xã', 'Diện tích (m2)', 'Loại hình', 'Giấy tờ pháp lý', 'Hướng', 'Chiều ngang (m)', 'Chiều dài (m)', 'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Tình trạng nội thất']


In [61]:
# Đọc file dữ liệu đã làm sạch
df = pd.read_csv('cleaned_nhatot_data.csv')

# Kiểm tra tên cột chính xác (trong file của bạn có thể khác)
print("Các cột trong DataFrame:")
print(df.columns.tolist())

# Tìm cột chứa giá (có thể tên là 'Giá bán' hoặc tương tự)
price_column = 'Giá bán'  # Điều chỉnh nếu tên cột khác

def convert_price_to_billion(price_str):
    """
    Chuyển đổi giá từ chuỗi sang số tỷ (float)
    - "3,5 tỷ" -> 3.5
    - "750 triệu" -> 0.75
    - "1.5 tỷ" -> 1.5
    - "1000000000" (số) -> 1.0
    """
    if pd.isna(price_str):
        return np.nan
    
    # Nếu đã là số, trả về tỷ (chia 1 tỷ)
    if isinstance(price_str, (int, float)):
        return price_str / 1_000_000_000
    
    # Chuyển sang string để xử lý
    price_str = str(price_str).strip()
    
    # Loại bỏ dấu chấm phân cách hàng nghìn (nếu có)
    price_str = price_str.replace('.', '')
    
    # Thay dấu phẩy bằng dấu chấm cho số thập phân
    price_str = price_str.replace(',', '.')
    
    # Chuyển đổi dựa trên từ khóa "tỷ" và "triệu"
    if 'tỷ' in price_str.lower():
        # Loại bỏ chữ "tỷ" và khoảng trắng
        value_str = price_str.lower().replace('tỷ', '').replace(' ', '')
        try:
            return float(value_str)
        except:
            return np.nan
    
    elif 'triệu' in price_str.lower():
        # Loại bỏ chữ "triệu" và khoảng trắng
        value_str = price_str.lower().replace('triệu', '').replace(' ', '')
        try:
            value = float(value_str)
            return value / 1000  # Chuyển triệu sang tỷ
        except:
            return np.nan
    
    else:
        # Nếu không có từ khóa, thử chuyển trực tiếp thành số
        try:
            # Loại bỏ tất cả ký tự không phải số, dấu chấm, dấu phẩy
            import re
            cleaned = re.sub(r'[^\d.,]', '', price_str)
            cleaned = cleaned.replace(',', '.')
            if cleaned == '':
                return np.nan
            value = float(cleaned)
            # Nếu số lớn hơn 1000, có thể đang là triệu
            if value > 1000:
                return value / 1_000_000_000
            else:
                return value
        except:
            return np.nan

# Áp dụng hàm chuyển đổi
df[price_column] = df[price_column].apply(convert_price_to_billion)

# Đổi tên cột để rõ ràng
df = df.rename(columns={price_column: 'Giá (tỷ VNĐ)'})

# Lưu lại file
df.to_csv('cleaned_nhatot_data.csv', index=False)

# Kiểm tra kết quả
print(f"\nĐã xử lý xong cột giá. Lưu file 'cleaned_nhatot_data.csv'")
print(f"Số dòng dữ liệu: {len(df)}")
print("\nThống kê giá (tỷ VNĐ):")
print(f"Min: {df['Giá (tỷ VNĐ)'].min():.2f}")
print(f"Max: {df['Giá (tỷ VNĐ)'].max():.2f}")
print(f"Mean: {df['Giá (tỷ VNĐ)'].mean():.2f}")
print(f"Median: {df['Giá (tỷ VNĐ)'].median():.2f}")


Các cột trong DataFrame:
['Giá bán', 'Thành phố', 'Phường/Xã', 'Diện tích (m2)', 'Loại hình', 'Giấy tờ pháp lý', 'Hướng', 'Chiều ngang (m)', 'Chiều dài (m)', 'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Tình trạng nội thất']

Đã xử lý xong cột giá. Lưu file 'cleaned_nhatot_data.csv'
Số dòng dữ liệu: 6396

Thống kê giá (tỷ VNĐ):
Min: 0.00
Max: 1250.00
Mean: 7.75
Median: 5.90


In [ ]:
import pandas as pd
df = pd.read_csv("cleaned_nhatot_data.csv")
nan_count = df.isna().sum()
nan_percent = nan_count / len(df) * 100
result = pd.DataFrame({
    "NaN_count": nan_count,
    "NaN_percent_%": nan_percent
})
print(result)

                     NaN_count  NaN_percent_%
Giá (tỷ VNĐ)                 1       0.015635
Thành phố                    0       0.000000
Phường/Xã                    1       0.015635
Diện tích (m2)               0       0.000000
Loại hình                    0       0.000000
Giấy tờ pháp lý              1       0.015635
Hướng                     4740      74.108818
Chiều ngang (m)           1211      18.933709
Chiều dài (m)             1497      23.405253
Số phòng ngủ                 0       0.000000
Số phòng vệ sinh          1517      23.717949
Số tầng                   2182      34.115072
Tình trạng nội thất       3057      47.795497


In [37]:
import pandas as pd
import numpy as np

# 1. LOAD DATA
df = pd.read_csv("cleaned_nhatot_data.csv")
print(f"Rows: {len(df):,} | Cols: {len(df.columns)}")

# 2. CHUẨN HÓA CỘT SỐ
for col in ['Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng',
            'Chiều ngang (m)', 'Chiều dài (m)', 'Diện tích (m2)']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. CHIỀU NGANG / CHIỀU DÀI
if {'Diện tích (m2)', 'Chiều ngang (m)', 'Chiều dài (m)'}.issubset(df.columns):
    # thiếu ngang
    m = df['Chiều ngang (m)'].isna() & df['Chiều dài (m)'].notna()
    df.loc[m, 'Chiều ngang (m)'] = df.loc[m, 'Diện tích (m2)'] / df.loc[m, 'Chiều dài (m)']

    # thiếu dài
    m = df['Chiều dài (m)'].isna() & df['Chiều ngang (m)'].notna()
    df.loc[m, 'Chiều dài (m)'] = df.loc[m, 'Diện tích (m2)'] / df.loc[m, 'Chiều ngang (m)']

    # thiếu cả hai
    valid = df.dropna(subset=['Chiều ngang (m)', 'Chiều dài (m)'])
    if len(valid) > 0:
        r = (valid['Chiều dài (m)'] / valid['Chiều ngang (m)']).median()
        m = df['Chiều ngang (m)'].isna() & df['Chiều dài (m)'].isna()
        df.loc[m, 'Chiều ngang (m)'] = np.sqrt(df.loc[m, 'Diện tích (m2)'] / r)
        df.loc[m, 'Chiều dài (m)'] = df.loc[m, 'Diện tích (m2)'] / df.loc[m, 'Chiều ngang (m)']

# 4. SỐ PHÒNG NGỦ ← DIỆN TÍCH
m = df['Số phòng ngủ'].isna() & df['Diện tích (m2)'].notna()
area_per_room = (df['Diện tích (m2)'] / df['Số phòng ngủ']).median()
df.loc[m, 'Số phòng ngủ'] = np.clip(
    (df.loc[m, 'Diện tích (m2)'] / area_per_room).round(), 1, None
)

# 5. WC ← PHÒNG NGỦ
wc_med = df.groupby('Số phòng ngủ')['Số phòng vệ sinh'].median()
m = df['Số phòng vệ sinh'].isna()
df.loc[m, 'Số phòng vệ sinh'] = df.loc[m, 'Số phòng ngủ'].map(wc_med)
df['Số phòng vệ sinh'].fillna(df['Số phòng vệ sinh'].median(), inplace=True)

# 6. SỐ TẦNG ← DIỆN TÍCH / PHÒNG NGỦ
m = df['Số tầng'].isna() & df['Diện tích (m2)'].notna() & df['Số phòng ngủ'].notna()
density = (df['Diện tích (m2)'] / df['Số phòng ngủ']).median()
df.loc[m, 'Số tầng'] = np.clip(
    (df.loc[m, 'Diện tích (m2)'] / df.loc[m, 'Số phòng ngủ']) / density,
    1, None
)
df['Số tầng'].fillna(df['Số tầng'].median(), inplace=True)

# 7. CATEGORICAL → MODE
for col in df.select_dtypes(include='object'):
    if df[col].isna().any():
        df[col].fillna(df[col].mode()[0], inplace=True)

# 8. SAVE
df.to_csv("data_nan_handled_final.csv", index=False)
print("✅ Hoàn tất xử lý NaN – đã lưu file.")

Rows: 6,396 | Cols: 13
✅ Hoàn tất xử lý NaN – đã lưu file.


C:\Users\LEGION\AppData\Local\Temp\ipykernel_15004\3333797857.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Số phòng vệ sinh'].fillna(df['Số phòng vệ sinh'].median(), inplace=True)
C:\Users\LEGION\AppData\Local\Temp\ipykernel_15004\3333797857.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

In [46]:
df = pd.read_csv("data_nan_handled_final.csv")
# 1. Giá_per_m2 = Giá / Diện tích
# Đảm bảo không chia cho 0
df['Giá_per_m2'] = np.where(
    df['Diện tích (m2)'] > 0,
    df['Giá (tỷ VNĐ)'] / df['Diện tích (m2)'],
    np.nan
)

# 2. Tổng_phòng = Phòng ngủ + Phòng vệ sinh
# Cộng và làm tròn số phòng
df['Tổng_phòng'] = (df['Số phòng ngủ'].fillna(0) + df['Số phòng vệ sinh'].fillna(0)).astype(int)

# 3. Aspect_ratio = Chiều ngang / Chiều dài
# Xử lý trường hợp chia cho 0
df['Aspect_ratio'] = np.where(
    df['Chiều dài (m)'] > 0,
    df['Chiều ngang (m)'] / df['Chiều dài (m)'],
    np.nan
)

# 4. Diện_tích_per_phòng = Diện tích / Tổng phòng
# Xử lý trường hợp tổng phòng = 0
df['Diện_tích_per_phòng'] = np.where(
    df['Tổng_phòng'] > 0,
    df['Diện tích (m2)'] / df['Tổng_phòng'],
    np.nan
)

# Lưu file mới
output_file = "data_with_new_features.csv"
df.to_csv(output_file, index=False)

# Kiểm tra thống kê các đặc trưng mới
print("\n📊 Thống kê các đặc trưng mới:")
print("1. Giá_per_m2 (tỷ VNĐ/m2):")
print(f"   Min: {df['Giá_per_m2'].min():.4f}")
print(f"   Max: {df['Giá_per_m2'].max():.4f}")
print(f"   Mean: {df['Giá_per_m2'].mean():.4f}")
print(f"   Median: {df['Giá_per_m2'].median():.4f}")

print("\n2. Tổng_phòng (phòng):")
print(df['Tổng_phòng'].value_counts().head(10))

print("\n3. Aspect_ratio (chiều ngang/chiều dài):")
print(f"   Min: {df['Aspect_ratio'].min():.2f}")
print(f"   Max: {df['Aspect_ratio'].max():.2f}")
print(f"   Mean: {df['Aspect_ratio'].mean():.2f}")
print(f"   Median: {df['Aspect_ratio'].median():.2f}")

print("\n4. Diện_tích_per_phòng (m2/phòng):")
print(f"   Min: {df['Diện_tích_per_phòng'].min():.2f}")
print(f"   Max: {df['Diện_tích_per_phòng'].max():.2f}")
print(f"   Mean: {df['Diện_tích_per_phòng'].mean():.2f}")
print(f"   Median: {df['Diện_tích_per_phòng'].median():.2f}")



📊 Thống kê các đặc trưng mới:
1. Giá_per_m2 (tỷ VNĐ/m2):
   Min: 0.0000
   Max: 0.9470
   Mean: 0.1157
   Median: 0.1047

2. Tổng_phòng (phòng):
Tổng_phòng
4     1303
6     1040
8      828
5      758
7      555
10     404
9      393
3      349
11     220
2      206
Name: count, dtype: int64

3. Aspect_ratio (chiều ngang/chiều dài):
   Min: 0.03
   Max: 2870.42
   Mean: 0.82
   Median: 0.31

4. Diện_tích_per_phòng (m2/phòng):
   Min: 1.50
   Max: 839.00
   Mean: 14.40
   Median: 9.60


In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu
df = pd.read_csv("data_with_new_features.csv")

# Chia 8/2 random
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42
)

# Lưu file
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

print(f"✅ Đã chia dữ liệu 8/2:")
print(f"  - Train: {train_df.shape} (80%)")
print(f"  - Test: {test_df.shape} (20%)")
print(f"  - Cột: {train_df.columns.tolist()}")

✅ Đã chia dữ liệu 8/2:
  - Train: (5116, 17) (80%)
  - Test: (1280, 17) (20%)
  - Cột: ['Giá (tỷ VNĐ)', 'Thành phố', 'Phường/Xã', 'Diện tích (m2)', 'Loại hình', 'Giấy tờ pháp lý', 'Hướng', 'Chiều ngang (m)', 'Chiều dài (m)', 'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Tình trạng nội thất', 'Giá_per_m2', 'Tổng_phòng', 'Aspect_ratio', 'Diện_tích_per_phòng']


In [50]:
import pandas as pd
import numpy as np

# ----- ĐỌC DỮ LIỆU -----
# Đọc file dữ liệu đã làm sạch (cột giá đã được xử lý)
df = pd.read_csv('test_data.csv')

# ----- KIỂM TRA DỮ LIỆU -----
print("=== THÔNG TIN DỮ LIỆU ===")
print(f"Số dòng: {len(df)}")
print(f"Số cột: {len(df.columns)}")
print("\nCác cột trong DataFrame:")
print(df.columns.tolist())
print("\n5 dòng đầu tiên:")
print(df.head())
print("\nThông tin các cột:")
print(df.info())

# Xác định cột mục tiêu (sau khi đã xử lý)
# Tìm cột giá (cột đã được xử lý từ code trước)
if 'Giá (tỷ VNĐ)' in df.columns:
    target_col = 'Giá (tỷ VNĐ)'
    print(f"\nCột mục tiêu: {target_col}")
else:
    # Nếu không tìm thấy, tìm cột có 'giá' trong tên
    for col in df.columns:
        if 'giá' in col.lower() or 'price' in col.lower():
            target_col = col
            print(f"\nCột mục tiêu: {target_col}")
            break
    else:
        print("\n⚠️ Cảnh báo: Không tìm thấy cột giá. Kiểm tra tên cột trong danh sách trên.")
        exit()

# ----- KIỂM TRA CÁC CỘT CẦN ENCODING -----
# Danh sách các cột cần encoding
categorical_cols = ['Thành phố', 'Phường/Xã', 'Loại hình', 'Giấy tờ pháp lý', 'Hướng', 'Tình trạng nội thất']

# Kiểm tra xem các cột có tồn tại không
missing_cols = []
available_cols = []
for col in categorical_cols:
    if col in df.columns:
        available_cols.append(col)
    else:
        missing_cols.append(col)

print(f"\n=== KIỂM TRA CÁC CỘT ===")
print(f"Các cột có sẵn: {available_cols}")
if missing_cols:
    print(f"Các cột không tồn tại: {missing_cols}")

# ----- Tạo DataFrame để lưu mapping -----
encoding_report = []

# ----- 1. Target Encoding với smoothing và gom nhóm -----
def target_encode_with_smoothing(df, col, target=target_col, min_samples=10, smoothing=10):
    try:
        # Kiểm tra nếu có giá trị NaN trong cột cần encode
        if df[col].isnull().any():
            print(f"⚠️ Cột '{col}' có giá trị NaN. Điền giá trị mặc định...")
            df[col] = df[col].fillna('Không xác định')
        
        # Thực hiện target encoding
        means = df.groupby(col)[target].mean()
        counts = df.groupby(col)[target].count()
        global_mean = df[target].mean()
        smooth_means = (means * counts + global_mean * smoothing) / (counts + smoothing)
        smooth_means.loc[counts < min_samples] = global_mean
        return df[col].map(smooth_means), smooth_means.to_dict()
    except Exception as e:
        print(f"❌ Lỗi khi encode {col}: {e}")
        # Trả về series với giá trị global_mean nếu có lỗi
        return pd.Series([global_mean] * len(df)), {}

# Thành phố (nếu tồn tại)
if 'Thành phố' in df.columns:
    df['Thành phố_encoded'], city_map = target_encode_with_smoothing(df, 'Thành phố', min_samples=5)
    for city, val in city_map.items():
        encoding_report.append({'Feature': 'Thành phố', 'Category': city, 'Encoded Value': val})
    print(f"✅ Đã encode Thành phố với {len(city_map)} categories")

# Phường/Xã (nếu tồn tại)
if 'Phường/Xã' in df.columns:
    df['Phường/Xã_encoded'], ward_map = target_encode_with_smoothing(df, 'Phường/Xã', min_samples=3)
    for ward, val in ward_map.items():
        encoding_report.append({'Feature': 'Phường/Xã', 'Category': ward, 'Encoded Value': val})
    print(f"✅ Đã encode Phường/Xã với {len(ward_map)} categories")

# ----- 2. Loại hình (One-Hot) -----
if 'Loại hình' in df.columns:
    # Xử lý giá trị NaN trong Loại hình
    df['Loại hình'] = df['Loại hình'].fillna('Không xác định')
    
    # Tạo one-hot encoding
    try:
        loai_hinh_dummies = pd.get_dummies(df['Loại hình'], prefix='Loại hình')
        df = pd.concat([df, loai_hinh_dummies], axis=1)
        # Lấy các cột one-hot đã tạo
        loai_hinh_cols = [col for col in df.columns if col.startswith('Loại hình_')]
        for col in loai_hinh_cols:
            category = col.replace('Loại hình_', '')
            encoding_report.append({'Feature': 'Loại hình', 'Category': category, 'Encoded Value': f'One-Hot: {col}'})
        print(f"✅ Đã tạo {len(loai_hinh_cols)} cột one-hot cho Loại hình")
    except Exception as e:
        print(f"❌ Lỗi khi one-hot encoding Loại hình: {e}")

# ----- 3. Giấy tờ pháp lý (Ordinal) -----
if 'Giấy tờ pháp lý' in df.columns:
    phap_ly_order = {
        'Đã có sổ': 4,
        'Sổ chung / công chứng vi bằng': 3,
        'Đang chờ sổ': 2,
        'Không có sổ': 1,
        'Giấy tờ viết tay': 0
    }
    
    # Xử lý giá trị NaN
    df['Giấy tờ pháp lý'] = df['Giấy tờ pháp lý'].fillna('Không xác định')
    
    # Thêm giá trị mặc định cho các category không có trong mapping
    unique_values = df['Giấy tờ pháp lý'].unique()
    for val in unique_values:
        if val not in phap_ly_order and val != 'Không xác định':
            phap_ly_order[val] = 2  # Giá trị mặc định
    
    if 'Không xác định' not in phap_ly_order:
        phap_ly_order['Không xác định'] = 2
    
    df['Giấy tờ pháp lý_encoded'] = df['Giấy tờ pháp lý'].map(phap_ly_order)
    for category, val in phap_ly_order.items():
        encoding_report.append({'Feature': 'Giấy tờ pháp lý', 'Category': category, 'Encoded Value': val})
    print(f"✅ Đã encode Giấy tờ pháp lý với {len(phap_ly_order)} categories")

# ----- 4. Hướng (Cyclical) -----
if 'Hướng' in df.columns:
    huong_order = {
        'Bắc': 0, 'Đông Bắc': 45, 'Đông': 90, 'Đông Nam': 135,
        'Nam': 180, 'Tây Nam': 225, 'Tây': 270, 'Tây Bắc': 315
    }
    
    # Xử lý giá trị NaN
    df['Hướng'] = df['Hướng'].fillna('Không xác định')
    
    # Thêm giá trị mặc định cho các category không có trong mapping
    unique_h_values = df['Hướng'].unique()
    for val in unique_h_values:
        if val not in huong_order and val != 'Không xác định':
            huong_order[val] = 0  # Giá trị mặc định
    
    if 'Không xác định' not in huong_order:
        huong_order['Không xác định'] = 0
    
    df['Hướng_goc'] = df['Hướng'].map(huong_order)
    df['Hướng_sin'] = np.sin(np.radians(df['Hướng_goc']))
    df['Hướng_cos'] = np.cos(np.radians(df['Hướng_goc']))
    
    for category, val in huong_order.items():
        encoding_report.append({'Feature': 'Hướng (gốc)', 'Category': category, 'Encoded Value': val})
    encoding_report.append({'Feature': 'Hướng', 'Category': 'sin transformation', 'Encoded Value': 'sin(radians(góc))'})
    encoding_report.append({'Feature': 'Hướng', 'Category': 'cos transformation', 'Encoded Value': 'cos(radians(góc))'})
    print(f"✅ Đã encode Hướng với {len(huong_order)} categories")

# ----- 5. Tình trạng nội thất (Ordinal) -----
if 'Tình trạng nội thất' in df.columns:
    noi_that_order = {
        'Nội thất cao cấp': 1.5,
        'Nội thất đầy đủ': 1.3,
        'Hoàn thiện cơ bản': 1.1,
        'Bàn giao thô': 1.0
    }
    
    # Xử lý giá trị NaN
    df['Tình trạng nội thất'] = df['Tình trạng nội thất'].fillna('Hoàn thiện cơ bản')
    
    # Thêm giá trị mặc định cho các category không có trong mapping
    unique_n_values = df['Tình trạng nội thất'].unique()
    for val in unique_n_values:
        if val not in noi_that_order:
            noi_that_order[val] = 1.1  # Giá trị mặc định
    
    df['Tình trạng nội thất_encoded'] = df['Tình trạng nội thất'].map(noi_that_order)
    for category, val in noi_that_order.items():
        encoding_report.append({'Feature': 'Tình trạng nội thất', 'Category': category, 'Encoded Value': val})
    print(f"✅ Đã encode Tình trạng nội thất với {len(noi_that_order)} categories")

# ----- Loại bỏ cột gốc (nếu tồn tại) -----
cols_to_drop = []
for col in ['Thành phố', 'Phường/Xã', 'Loại hình', 'Giấy tờ pháp lý', 'Hướng', 'Tình trạng nội thất', 'Hướng_goc']:
    if col in df.columns:
        cols_to_drop.append(col)

if cols_to_drop:
    df = df.drop(columns=cols_to_drop)
    print(f"\n✅ Đã xóa các cột gốc: {cols_to_drop}")

# ----- Xem vài dòng đã encode -----
print(f"\n=== 5 dòng đầu sau encoding ===")
print(df.head().to_string())
print(f"\nShape sau encoding: {df.shape}")

# ----- LƯU DỮ LIỆU -----
output_file = 'encoded_nhatot_data_test.csv'
df.to_csv(output_file, index=False)
print(f"\n✅ Đã lưu dữ liệu đã encode vào: {output_file}")


=== THÔNG TIN DỮ LIỆU ===
Số dòng: 1280
Số cột: 17

Các cột trong DataFrame:
['Giá (tỷ VNĐ)', 'Thành phố', 'Phường/Xã', 'Diện tích (m2)', 'Loại hình', 'Giấy tờ pháp lý', 'Hướng', 'Chiều ngang (m)', 'Chiều dài (m)', 'Số phòng ngủ', 'Số phòng vệ sinh', 'Số tầng', 'Tình trạng nội thất', 'Giá_per_m2', 'Tổng_phòng', 'Aspect_ratio', 'Diện_tích_per_phòng']

5 dòng đầu tiên:
   Giá (tỷ VNĐ)       Thành phố                                 Phường/Xã  \
0          6.86          Hà Nội                           Phường Giáp Bát   
1          5.28  Tp Hồ Chí Minh                      Phường Tân Thới Nhất   
2          5.90  Tp Hồ Chí Minh                                 Phường 12   
3          6.50  Tp Hồ Chí Minh                          Phường Tân Kiểng   
4          1.68  Tp Hồ Chí Minh  Phường Hiệp Bình Chánh (Quận Thủ Đức cũ)   

   Diện tích (m2)        Loại hình Giấy tờ pháp lý     Hướng  Chiều ngang (m)  \
0            36.0     Nhà ngõ, hẻm        Đã có sổ  Đông Nam         3.340659   
1    

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280 entries, 0 to 1279
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Giá (tỷ VNĐ)                     1280 non-null   float64
 1   Diện tích (m2)                   1280 non-null   float64
 2   Chiều ngang (m)                  1280 non-null   float64
 3   Chiều dài (m)                    1280 non-null   float64
 4   Số phòng ngủ                     1280 non-null   float64
 5   Số phòng vệ sinh                 1280 non-null   float64
 6   Số tầng                          1280 non-null   float64
 7   Giá_per_m2                       1280 non-null   float64
 8   Tổng_phòng                       1280 non-null   int64  
 9   Aspect_ratio                     1280 non-null   float64
 10  Diện_tích_per_phòng              1280 non-null   float64
 11  Thành phố_encoded                1280 non-null   float64
 12  Phường/Xã_encoded   